In [17]:
from TelescopeML.CNN_Transformer import *
from TelescopeML.DataMaster import *

In [18]:
import os 

__reference_data_path__ = os.getenv("TelescopeML_reference_data")
__reference_data_path__ 


# Note: insert the directory of the reference_data if you get an error reading the reference data!!!
# __reference_data_path__ = 'INSERT_DIRECTORY_OF_reference_data'


'/Users/danielmittelman/TelescopeML_project/reference_data'

In [19]:
train_BD = pd.read_csv(os.path.join(__reference_data_path__, 
                                    'training_datasets', 
                                    'browndwarf_R100_v4_newWL_v3.csv.bz2'), compression='bz2')
train_BD.head()

,gravity,temperature,c_o_ratio,metallicity,2.512,2.487,2.462,2.438,2.413,2.389,...,0.981,0.971,0.962,0.952,0.943,0.933,0.924,0.915,0.906,0.897
0,5.0,1100,0.25,-1.0,9.103045e-08,1.181658e-07,1.307868e-07,1.269229e-07,1.159179e-07,8.925110e-08,...,1.257751e-07,9.640859e-08,7.612550e-08,6.901364e-08,6.247359e-08,4.112384e-08,5.127995e-08,4.897355e-08,4.087795e-08,2.791689e-08
1,5.0,1100,0.25,-0.7,9.103045e-08,1.181658e-07,1.307868e-07,1.269229e-07,1.159179e-07,8.925110e-08,...,1.257751e-07,9.640859e-08,7.612550e-08,6.901364e-08,6.247359e-08,4.112384e-08,5.127995e-08,4.897355e-08,4.087795e-08,2.791689e-08
2,5.0,1100,0.25,-0.5,9.103045e-08,1.181658e-07,1.307868e-07,1.269229e-07,1.159179e-07,8.925110e-08,...,1.257751e-07,9.640859e-08,7.612550e-08,6.901364e-08,6.247359e-08,4.112384e-08,5.127995e-08,4.897355e-08,4.087795e-08,2.791689e-08
3,5.0,1100,0.25,-0.3,9.103045e-08,1.181658e-07,1.307868e-07,1.269229e-07,1.159179e-07,8.925110e-08,...,1.257751e-07,9.640859e-08,7.612550e-08,6.901364e-08,6.247359e-08,4.112384e-08,5.127995e-08,4.897355e-08,4.087795e-08,2.791689e-08
4,5.0,1100,0.25,0.0,9.103045e-08,1.181658e-07,1.307868e-07,1.269229e-07,1.159179e-07,8.925110e-08,...,1.257751e-07,9.640859e-08,7.612550e-08,6.901364e-08,6.247359e-08,4.112384e-08,5.127995e-08,4.897355e-08,4.087795e-08,2.791689e-08


In [20]:
output_names = ['gravity', 'temperature', 'c_o_ratio', 'metallicity']
train_BD[output_names].head()

,gravity,temperature,c_o_ratio,metallicity
0,5.0,1100,0.25,-1.0
1,5.0,1100,0.25,-0.7
2,5.0,1100,0.25,-0.5
3,5.0,1100,0.25,-0.3
4,5.0,1100,0.25,0.0


In [21]:
wavelength_names = [item for item in train_BD.columns.to_list() if item not in output_names]
wavelength_names[:5]

['2.512', '2.487', '2.462', '2.438', '2.413']

In [22]:
wavelength_values = [float(item) for item in wavelength_names]
wavelength_values[:10]

[2.512, 2.487, 2.462, 2.438, 2.413, 2.389, 2.366, 2.342, 2.319, 2.296]

In [23]:
wl_synthetic = pd.read_csv(os.path.join(__reference_data_path__, 
                                        'training_datasets', 
                                        'wl.csv'))
wl_synthetic.head()

,wl
0,2.511960
1,2.486966
2,2.462220
3,2.437720
4,2.413464


In [24]:
# Training  variables
X = train_BD.drop(
    columns=['gravity', 
             'temperature', 
             'c_o_ratio', 
             'metallicity'])


# Target/Output feature variables
y = train_BD[['gravity', 'c_o_ratio', 'metallicity', 'temperature', ]]


In [25]:
y.loc[:, 'temperature'] = np.log10(y['temperature'])

In [26]:
data_processor = DataProcessor( 
                                 flux_values=X.to_numpy(),
                                 wavelength_names=X.columns,
                                 wavelength_values=wavelength_values,
                                 output_values=y.to_numpy(),
                                 output_names=output_names,
                                 spectral_resolution=200,
                                )

In [27]:
data_processor.split_train_validation_test(test_size=0.1, 
                                             val_size=0.1, 
                                             random_state_=42,)

In [28]:
data_processor.standardize_X_column_wise()
data_processor.standardize_y_column_wise()

In [29]:
# Example input shape: (sequence_length, num_features)
sequence_length = 104
num_features = 1  # e.g., univariate time series

model = build_cnn_transformer_model(
    input_shape=(sequence_length, num_features),
    num_cnn_filters=64,
    cnn_kernel_size=3,
    cnn_stride=1,
    cnn_padding='same',
    num_transformer_blocks=2,
    num_heads=4,
    transformer_ff_dim=128,
    dropout_rate=0.1,
    output_dim=4
)

model.compile(optimizer='adam',
                loss='mse', 
                metrics=['mae'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 104, 1)]             0         []                            
                                                                                                  
 conv1d_5 (Conv1D)           (None, 104, 64)              256       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_5 (Bat  (None, 104, 64)              256       ['conv1d_5[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 conv1d_6 (Conv1D)           (None, 104, 128)             24704     ['batch_normalization_5[

In [30]:
X_train = data_processor.X_train_standardized_columnwise
y_train = data_processor.y_train_standardized_columnwise

X_test = data_processor.X_test_standardized_columnwise
y_test = data_processor.y_test_standardized_columnwise

In [31]:
print(X_train.shape)
print(y_train.shape)

(25019, 104)
(25019, 4)


In [32]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]
)

Epoch 1/50
782/782 [==============================] - 107s 135ms/step - loss: 0.7949 - mae: 0.7361 - val_loss: 0.8993 - val_mae: 0.7861
Epoch 2/50
782/782 [==============================] - 106s 136ms/step - loss: 0.7239 - mae: 0.6922 - val_loss: 0.6545 - val_mae: 0.6465
Epoch 3/50
782/782 [==============================] - 106s 135ms/step - loss: 0.6784 - mae: 0.6641 - val_loss: 0.6191 - val_mae: 0.6223
Epoch 4/50
782/782 [==============================] - 9173s 12s/step - loss: 0.6537 - mae: 0.6475 - val_loss: 0.7114 - val_mae: 0.6941
Epoch 5/50
782/782 [==============================] - 105s 135ms/step - loss: 0.6402 - mae: 0.6391 - val_loss: 0.6252 - val_mae: 0.6296
Epoch 6/50
782/782 [==============================] - 249s 319ms/step - loss: 0.6300 - mae: 0.6327 - val_loss: 0.5839 - val_mae: 0.5972
Epoch 7/50
311/782 [==========>...................] - ETA: 1:00 - loss: 0.6414 - mae: 0.6391